In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import re

from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TERRAIN
from ast import literal_eval
from scipy import stats
from math import radians, sin, cos, acos, log, pi, tan, asin,sqrt
from decimal import Decimal
from bokeh.embed import components
from bokeh.resources import CDN
from bokeh.transform import factor_cmap
from bokeh.palettes import Category20b_20
from bokeh.models import ColumnDataSource, HoverTool

In [2]:
def merc(Coords):
    Coordinates = (Coords)
    lat = Coordinates[0]
    lon = Coordinates[1]
    
    r_major = 6378137.000
    x = r_major * radians(lon)
    scale = x/lon
    y = 180.0/pi * log(tan(pi/4.0 + 
        lat * (pi/180.0)/2.0)) * scale
    return (x, y)

In [3]:
airbnb_cr = pd.read_csv('/home/novais/Downloads/airbnb_cr.csv')

In [4]:
airbnb_cr.crimes_5km_count=airbnb_cr.crimes_1km_count+airbnb_cr.crimes_2km_count+airbnb_cr.crimes_5km_count
airbnb_cr.crimes_2km_count=airbnb_cr.crimes_1km_count+airbnb_cr.crimes_2km_count
airbnb_cr.head()

,Unnamed: 0,listing_id,date,price,month,day,year,host_id,neighbourhood,room_type,...,date_id,crimes_loc_1km,crimes_loc_2km,crimes_loc_5km,crimes_1km,crimes_2km,crimes_5km,crimes_1km_count,crimes_2km_count,crimes_5km_count
0,0,2384,2018-07-26,75.0,7,26,2018,2613,Hyde Park,2,...,17,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...","[244, 246]","[224, 225, 264, 265, 266, 633, 223]","[262, 263, 279, 281, 162, 163, 164, 165, 166, ...",4,18,107
1,268,2384,2018-07-29,69.0,7,29,2018,2613,Hyde Park,2,...,20,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...",[],"[224, 225, 265, 266, 267, 243, 223]","[262, 263, 278, 279, 280, 281, 289, 162, 163, ...",0,20,96
2,847,2384,2018-08-05,65.0,8,5,2018,2613,Hyde Park,2,...,27,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...",[246],"[265, 266, 267, 243, 223]","[262, 263, 280, 281, 289, 162, 163, 164, 165, ...",1,13,107
3,1343,2384,2018-08-12,75.0,8,12,2018,2613,Hyde Park,2,...,34,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...","[244, 246]","[224, 225, 265, 266, 267, 223]","[263, 278, 279, 289, 162, 163, 164, 165, 166, ...",3,17,101
4,1976,2384,2018-10-01,65.0,10,1,2018,2613,Hyde Park,2,...,84,"[244, 245, 246]","[223, 224, 225, 243, 264, 265, 266, 267, 632, ...","[162, 163, 164, 165, 166, 178, 179, 180, 181, ...",[244],"[224, 264, 265, 266, 223]","[263, 279, 280, 281, 289, 162, 163, 164, 165, ...",1,8,94


In [5]:
ids=list(airbnb_cr['listing_id'].unique())

In [6]:
loc_sum=airbnb_cr.groupby(['loc_id']).sum()
loc_sum=loc_sum.reset_index()
loc_sum=loc_sum[['loc_id','crimes_1km_count','crimes_2km_count','crimes_5km_count']]
loc_sum=loc_sum.rename(index=str, columns={"crimes_1km_count": "crimes_1km_sum","crimes_2km_count": "crimes_2km_sum","crimes_5km_count": "crimes_5km_sum"})
loc_sum.head()

,loc_id,crimes_1km_sum,crimes_2km_sum,crimes_5km_sum
0,0,9,14,21
1,1,7,23,110
2,2,6,100,492
3,3,64,232,1409
4,4,124,395,2277


In [7]:
loc_mean=airbnb_cr.groupby(['loc_id']).mean()
loc_mean=loc_mean.reset_index()
loc_mean=loc_mean[['loc_id','crimes_1km_count','crimes_2km_count','crimes_5km_count']]
loc_mean=loc_mean.rename(index=str, columns={"crimes_1km_count": "crimes_1km_mean","crimes_2km_count": "crimes_2km_mean","crimes_5km_count": "crimes_5km_mean"})
loc_mean.head()

,loc_id,crimes_1km_mean,crimes_2km_mean,crimes_5km_mean
0,0,1.500000,2.333333,3.500000
1,1,0.411765,1.352941,6.470588
2,2,0.375000,6.250000,30.750000
3,3,1.454545,5.272727,32.022727
4,4,1.937500,6.171875,35.578125


In [8]:
loc_count=loc_mean.merge(loc_sum,on='loc_id')
loc_count.head()

,loc_id,crimes_1km_mean,crimes_2km_mean,crimes_5km_mean,crimes_1km_sum,crimes_2km_sum,crimes_5km_sum
0,0,1.500000,2.333333,3.500000,9,14,21
1,1,0.411765,1.352941,6.470588,7,23,110
2,2,0.375000,6.250000,30.750000,6,100,492
3,3,1.454545,5.272727,32.022727,64,232,1409
4,4,1.937500,6.171875,35.578125,124,395,2277


In [9]:
airbnb_cr2=airbnb_cr.merge(loc_count,on='loc_id')
airbnb_cr2=airbnb_cr2.groupby('loc_id').mean()
airbnb_cr2=airbnb_cr2.reset_index()
airbnb_cr2=airbnb_cr2[['lat','lon','loc_id','crimes_1km_mean','crimes_2km_mean','crimes_5km_mean','crimes_1km_sum','crimes_2km_sum','crimes_5km_sum']]
airbnb_cr2['loc']=list(zip(airbnb_cr2.lat,airbnb_cr2.lon))
airbnb_cr2.head()

,lat,lon,loc_id,crimes_1km_mean,crimes_2km_mean,crimes_5km_mean,crimes_1km_sum,crimes_2km_sum,crimes_5km_sum,loc
0,41.65,-87.54,0,1.500000,2.333333,3.500000,9.0,14.0,21.0,"(41.65, -87.54)"
1,41.66,-87.55,1,0.411765,1.352941,6.470588,7.0,23.0,110.0,"(41.65999999999998, -87.54999999999997)"
2,41.67,-87.66,2,0.375000,6.250000,30.750000,6.0,100.0,492.0,"(41.67, -87.66000000000001)"
3,41.69,-87.68,3,1.454545,5.272727,32.022727,64.0,232.0,1409.0,"(41.69000000000004, -87.67999999999995)"
4,41.69,-87.67,4,1.937500,6.171875,35.578125,124.0,395.0,2277.0,"(41.69000000000004, -87.67000000000004)"


In [10]:
# listings2['loc'] = listings2['loc'].apply(lambda x: literal_eval(x))
airbnb_cr2['coords_x'] = airbnb_cr2['loc'].apply(lambda x: merc(x)[0])
airbnb_cr2['coords_y'] = airbnb_cr2['loc'].apply(lambda x: merc(x)[1])

p1 = figure(title = "Listings and Crimes")
p1.xaxis.axis_label = 'Latitude'
p1.yaxis.axis_label = 'Longitude'

a1 = airbnb_cr2[['crimes_1km_sum','crimes_1km_mean','coords_x','coords_y']]
a2 = airbnb_cr2[['crimes_2km_sum','crimes_2km_mean','coords_x','coords_y']]
a5 = airbnb_cr2[['crimes_5km_sum','crimes_5km_mean','coords_x','coords_y']]

source5 = ColumnDataSource(data=dict(
    s=a5['crimes_5km_sum'],
    cx=a5['coords_x'],
    cy=a5['coords_y'],
    sizes = [50]*len(a5),
    names = ['5 km']*len(a5),
    colors = ['#F39A05']*len(a5)
    ))
source2 = ColumnDataSource(data=dict(
    s=a2['crimes_2km_sum'],
    cx=a2['coords_x'],
    cy=a2['coords_y'],
    sizes = [30]*len(a2),
    names = ['2 km']*len(a2),
    colors = ['#D34B08']*len(a2)
    ))
source1 = ColumnDataSource(data=dict(
    s=a1['crimes_1km_sum'],
    cx=a1['coords_x'],
    cy=a1['coords_y'],
    sizes = [10]*len(a1),
    names = ['1 km']*len(a1),
    colors = ['#B70B0D']*len(a1)
    ))



p1 = figure(x_axis_type="mercator", y_axis_type="mercator")
p1.add_tile(CARTODBPOSITRON)

p1.circle(x = 'cx',y = 'cy',source=source5,size='sizes',legend ='names',color='colors',fill_alpha=0.5)
p1.circle(x = 'cx',y = 'cy',source=source2,size='sizes',legend ='names',color='colors',fill_alpha=0.6)
p1.circle(x = 'cx',y = 'cy',source=source1,size='sizes',legend ='names',color='colors',fill_alpha=0.7)



p1.add_tools(HoverTool(
    tooltips=[
        ( 'Nº of occurrences', '@s')
    ],
    
    formatters={
        'Nº of occurrences' : 'numeral',   
    },

))

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

p1_script, p1_div = components(p1)

In [11]:
output_notebook()


Loading BokehJS ...

In [13]:
show(p1)

In [14]:
print(p1_script)


<script type="text/javascript">
  (function() {
    var fn = function() {
      Bokeh.safely(function() {
        (function(root) {
          function embed_document(root) {
            
          var docs_json = '{"4dfb8efc-46cf-4992-a445-9c35cad1ed9b":{"roots":{"references":[{"attributes":{"plot":{"id":"1042","subtype":"Figure","type":"Plot"},"ticker":{"id":"1052","type":"MercatorTicker"}},"id":"1059","type":"Grid"},{"attributes":{"formatter":{"id":"1063","type":"MercatorTickFormatter"},"plot":{"id":"1042","subtype":"Figure","type":"Plot"},"ticker":{"id":"1061","type":"MercatorTicker"}},"id":"1060","type":"MercatorAxis"},{"attributes":{"dimension":"lat"},"id":"1061","type":"MercatorTicker"},{"attributes":{"fill_alpha":{"value":0.6},"fill_color":{"field":"colors"},"line_color":{"field":"colors"},"size":{"field":"sizes","units":"screen"},"x":{"field":"cx"},"y":{"field":"cy"}},"id":"1101","type":"Circle"},{"attributes":{"dimension":"lat"},"id":"1063","type":"MercatorTickFormatter"},{"a

In [15]:
print(p1_div)


<div class="bk-root" id="da0db949-1dd8-4e82-aec4-a3e3d2246de7"></div>
